In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import product
from scipy.stats import laplace

df = pd.read_csv("movie_metadata.csv")
for i in df.index:
    if(df.isna().at[i,"gross"] or df.isna().at[i,"language"] or df.isna().at[i,"country"]): df = df.drop(i)

In [2]:
def topgross(df,eplison):
    if(df.shape[0] == 1): sensibilidade = 0 
    else : sensibilidade = np.max(df["gross"].values) - np.max(np.delete(df["gross"].values,np.where(df["gross"].values == [np.max(df["gross"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["gross"].values) + x

In [3]:
def score_gross(df,eplison):
    Dict = {}
    
    #max_value = topgross(df,eplison)    
    #A = [df["movie_title"].values,df["movie_title"].values]
    #A = list(product(*A))
    
    #for mov1, mov2 in A:
    #    if(mov1 == mov2):
    #        value = df[df["movie_title"] == mov1]["gross"].values[0]
    #        Dict[mov1,mov2] = -abs(max_value - value)/max_value
    #    else: Dict[mov1,mov2] = 0
    
    for movie_title in df["movie_title"].values:
        value = df[df["movie_title"] == movie_title]["gross"].values[0]
        Dict[movie_title] = -abs(max_value - value)/max_value
        
    return Dict

In [4]:
def consulta1(df, eplison):
     
    fscore_df = score_gross(df,eplison)  
    sensibilidade = max(fscore_df.values())*(-1)
    
    total_sum = 0
    for movie_title in df["movie_title"].values:
        total_sum += np.e**(eplison*fscore_df[movie_title]/(2*sensibilidade))

    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(eplison*fscore_df[df.at[idnx,"movie_title"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "movie_title"]

In [5]:
def consulta2(df, eplison):
    result = []
        
    language_set = set(df["language"].values)
    for language in language_set:
        df_language = df[df["language"] == language]
        result.append(consulta1(df_language,eplison))
    
    return result

In [6]:
def topcount_country(df, eplison):
    group_by = df.groupby(["country"], as_index=False).count()
    if(df.shape[0] == 1): sensibilidade = 0
    else: sensibilidade = np.max(df["color"].values) - np.max(np.delete(df["color"].values,np.where(df["color"].values == [np.max(df["color"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["color"].values) + x

In [7]:
def score_count(df, eplison):
    Dict = {}
    group_by = df.groupby(["country"], as_index=False).count()
    max_value = topcount_country(group_by, eplison)
    
    for movie_country in df["country"].values:
        Dict[movie_country] = -abs(max_value-group_by[group_by["country"]== movie_country]["color"].values[0])/max_value
        
    return Dict

In [18]:
def consulta3_aux(df, eplison):
    
    fscore_df = score_count(df,eplison)  
    sensibilidade = max(fscore_df.values())*(-1)
    
    
    total_sum = 0
    for movie_country in df["country"].values:
        total_sum += np.e**(eplison*fscore_df[movie_country]/(2*sensibilidade))
    
   
    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(eplison*fscore_df[df.at[idnx,"country"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "country"]
    

In [29]:
def consulta3(df, eplison):
    result = []
    r1 = consulta3_aux(df, eplison/3)
    result.append(r1)
    r2 = consulta3_aux(df[df["country"]!=r1], eplison/3)
    result.append(r2)
    r3 = consulta3_aux(df[df["country"]!=r1][df[df["country"]!=r1]["country"]!=r2], eplison/3)
    result.append(r3)
    
    return result